In [1]:
import numpy as np
import pandas as pd
import os
import glob
import superskrypt as sp
from scipy import stats

In [2]:
PTH = r"C:\Users\LEON-NETB\Dropbox\LEON_CIECHANOWSKI\DANE_EEG"
os.chdir(PTH)
print(os.getcwd())

C:\Users\LEON-NETB\Dropbox\LEON_CIECHANOWSKI\DANE_EEG


In [3]:
#for f in glob.glob('*.txt'):
#     print(f)

In [4]:
def is_odd(num):
    return num % 2

In [5]:
# counts SoA and RT for all subjects and concatenates them in a DataFrame 

db_ALL = pd.DataFrame()
d2_all = pd.DataFrame()
subj_num = []
for f in glob.glob('*.txt'):
    subj_num.append(int(f.split('.')[0][1:]))
    df = pd.read_csv(f, sep='\t')
    print('read {}!'.format(f))
    db_SoA = sp.get_color_info(df)
    db_RT = sp.get_RT(df)
    db_ALL = db_ALL.append(db_SoA, ignore_index=True)
    for col in db_RT.columns:
        db_ALL.loc[0, '{}'.format(col)] = db_RT.loc[1, col]
    d2_all = d2_all.append(db_SoA, ignore_index=True)
print('DONE!')

read s2.txt!
DONE!


D:\Miniconda3\envs\py3\lib\site-packages\pandas\core\indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [7]:
# writing means of each condition (comp_l... etc):

for c in db_ALL.columns:
    for i in db_ALL.index:
        db_ALL.loc[i, c] = np.mean(db_ALL.loc[i, c])

In [8]:
# means of conditions of both hands:

for c in db_ALL.columns:
    for i in db_ALL.index:
        db_ALL.loc[i, 'comp'] = (db_ALL.loc[i, 'comp_l']+db_ALL.loc[i, 'comp_r'])/2
        db_ALL.loc[i, 'incomp'] = (db_ALL.loc[i, 'incomp_l']+db_ALL.loc[i, 'incomp_r'])/2
        db_ALL.loc[i, 'neut'] = (db_ALL.loc[i, 'neut_l']+db_ALL.loc[i, 'neut_r'])/2

In [9]:
db_ALL.index = subj_num
for i in db_ALL.index:
    if is_odd(i):
        db_ALL.loc[i, 'condition'] = 'info'
    else:
        db_ALL.loc[i, 'condition'] = 'noinfo'

In [10]:
# Derives all SoA for each codition (comp/neut/incomp), 
# each hand (l/r) and each block (1-6) separately
# and makes a separate column for them

for col in d2_all.columns:
    for i in range(0,6):
        db_ALL.loc[:, '{}_{}'.format(col, i+1)] = d2_all.loc[0, col][i]

In [13]:
db_ALL = db_ALL.loc[np.sort(db_ALL.index), :]
db_ALL.to_excel('SoA_EEG.xlsx')